# Assignment 1-2: Linear Classifiers

In [ ]:
%load_ext autoreload
%autoreload 2

# Data preprocessing

## Setup code

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import statistics
import random
import time
import math
%matplotlib inline

import utils


plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['font.size'] = 16

## Load the CIFAR-10 dataset

아래 코드는 `utils.data.preprocess` 함수를 통해 데이터셋을 로드합니다.

In [ ]:
import utils

utils.reset_seed(0)
data_dict = utils.preprocess_cifar10(cuda=False, dtype=torch.float64)
print('Train data shape: ', data_dict['X_train'].shape)
print('Train labels shape: ', data_dict['y_train'].shape)
print('Validation data shape: ', data_dict['X_val'].shape)
print('Validation labels shape: ', data_dict['y_val'].shape)
print('Test data shape: ', data_dict['X_test'].shape)
print('Test labels shape: ', data_dict['y_test'].shape)

# Softmax cross-entropy Loss 기반 Linear Classifier

본 과제에서는 아래와 같은 내용을 다루게 됩니다
- Softmax 분류기의 **loss function**을 구현한다.  
- **Gradient**의 해석적 표현(analytic)을 완전 벡터화로 구현한다.  
- 수치 미분과 비교해 **구현을 검증**한다.  
- Test dataset을 사용해 **learning rate**와 **regularization** 강도를 **tuning**한다.  
- **SGD**를 사용해 **loss function**을 **최적화**한다.  
- 최종적으로 학습된 **weights**를 **시각화**한다.  

Note: `.to()`나 `.cuda()`와 같은 메소드를 사용하지 마세요. 이 경우 의도치 않은 에러가 발생할 수 있습니다.

### 구현 하기

먼저 `softmax_loss_naive` 함수에서 **중첩 loop**를 사용해 naive한 softmax cross-entropy **loss function**을 구현합니다.  
구현한 **loss**가 올바른지 간단히 확인(sanity check)하기 위해 아래 코드는 작은 랜덤 **weight** 행렬과 regularization을 사용하지 않은 상태에서 softmax classifier를 실행합니다.<br>
이때 출력되는 **loss**는 log(10) ≈ 2.3 근처여야 합니다.  

In [ ]:
import utils
from linear_classifier import softmax_loss_naive

utils.reset_seed(0)
# Generate a random softmax weight tensor and use it to compute the loss.
W = 0.0001 * torch.randn(3072, 10, dtype=data_dict['X_val'].dtype, device=data_dict['X_val'].device)

X_batch = data_dict['X_val'][:128]
y_batch = data_dict['y_val'][:128]

loss, _ = softmax_loss_naive(W, X_batch, y_batch, reg=0.0)

# As a rough sanity check, our loss should be something close to log(10.0).
print('loss: %f' % loss)
print('sanity check: %f' % (math.log(10.0)))

### 구현 하기

위에서 함수가 반환한 `_grad_`는 현재 전부 0입니다.  
`softmax_loss_naive` 함수 안의 TODO 블록을 채워 넣어 softmax cross-entropy loss function의 **gradient**를 유도하고, 그 식을 함수 안에 직접 구현하세요.  
기존 코드 흐름 속에 새 코드를 적절히 섞어 넣는 방식이 도움이 됩니다.

아래 코드 셀은 구현한 gradient 계산이 맞는지 확인하기 위해 numeric gradient checking을 사용합니다.<br>
즉, finite differences로 forward pass의 gradient를 numerical하게 근사하고, 이 값과 여러분이 구현한 analytic gradient를 비교합니다.

Gradient 계산을 제대로 구현했으면 아래 실행되는 셀에서 **relative error**가 `1e-5`보다 작게 나와야 합니다.

In [ ]:
import utils
from linear_classifier import softmax_loss_naive

utils.reset_seed(0)
W = 0.0001 * torch.randn(3072, 10, dtype=data_dict['X_val'].dtype, device=data_dict['X_val'].device)
batch_size = 64
X_batch = data_dict['X_val'][:batch_size]
y_batch = data_dict['y_val'][:batch_size]

# Compute the loss and its gradient at W.
# TODO: implement the gradient part of 'softmax_loss_naive' function in "linear_classifier.py"
_, grad = softmax_loss_naive(W, X_batch, y_batch, reg=0.0)

# Numerically compute the gradient along several randomly chosen dimensions, and
# compare them with your analytically computed gradient. The numbers should
# match almost exactly along all dimensions.
f = lambda w: softmax_loss_naive(w, X_batch, y_batch, reg=0.0)[0]
grad_numerical = utils.grad_check_sparse(f, W, grad)

아래 코드 셀은 regularization을 켠 상태에서 gradient check를 합니다.<br>
마찬가지로 relative errors는 `1e-5` 미만이어야 합니다.

In [ ]:
import utils
from linear_classifier import softmax_loss_naive

utils.reset_seed(0)
W = 0.0001 * torch.randn(3072, 10, dtype=data_dict['X_val'].dtype, device=data_dict['X_val'].device)
batch_size = 64
X_batch = data_dict['X_val'][:batch_size]
y_batch = data_dict['y_val'][:batch_size]

# Compute the loss and its gradient at W.
# TODO: check your 'softmax_loss_naive' implementation with different 'reg'
_, grad = softmax_loss_naive(W, X_batch, y_batch, reg=1e3) 

# Numerically compute the gradient along several randomly chosen dimensions, and
# compare them with your analytically computed gradient. The numbers should
# match almost exactly along all dimensions.
f = lambda w: softmax_loss_naive(w, X_batch, y_batch, reg=1e3)[0]
grad_numerical = utils.grad_check_sparse(f, W, grad)

### 구현 하기

이제 **vectorized** 버전 `softmax_loss_vectorized`를 구현합니다.<br>
기존에 구현한 naive 버전과 vectorized 버전의 차이점은 **명시적인 loop 없이 구현**을 해야한다는 점입니다.<br>
입출력은 앞서 만든 naive 버전과 동일해야 합니다.

잘 구현이 되었다면 수치적인 오차는 (결과 값에 차이가 있을수 있으나) `1e-6` 이하여야 합니다.<br>
하지만 non-vectorized 버전과 vectorized 버전의 speed는 약 3\~4배 정도 차이나게 됩니다.<br>
GPU에서 실행하는 경우 15\~20x speed 향상을 기대할 수 있지만 본 과제는 CPU에서 실행하기 때문에 vectorized 구현이 그렇게 차이가 많이 나지는 않습니다.

In [ ]:
import utils
from linear_classifier import softmax_loss_naive, softmax_loss_vectorized

utils.reset_seed(0)
W = 0.0001 * torch.randn(3072, 10, dtype=data_dict['X_val'].dtype, device=data_dict['X_val'].device)
reg = 0.05

X_batch = data_dict['X_val'][:128]
y_batch = data_dict['y_val'][:128]

# Run and time the naive version
tic = time.time()
loss_naive, grad_naive = softmax_loss_naive(W, X_batch, y_batch, reg)
toc = time.time()
ms_naive = 1000.0 * (toc - tic)
print('naive loss: %e computed in %fs' % (loss_naive, ms_naive))

# Run and time the vectorized version
# TODO: Complete the implementation of softmax_loss_vectorized
tic = time.time()
loss_vec, grad_vec = softmax_loss_vectorized(W, X_batch, y_batch, reg)
toc = time.time()
ms_vec = 1000.0 * (toc - tic)
print('vectorized loss: %e computed in %fs' % (loss_vec, ms_vec))

# we use the Frobenius norm to compare the two versions of the gradient.
loss_diff = (loss_naive - loss_vec).abs().item()
grad_diff = torch.norm(grad_naive - grad_vec, p='fro')
print('Loss difference: %.2e' % loss_diff)
print('Gradient difference: %.2e' % grad_diff)
print('Speedup: %.2fX' % (ms_naive / ms_vec))

### 구현 하기

이제 loss function과 gradient 계산을 vectorized 방식으로 구현했으므로 이를 활용해 선형 분류기(linear classifier)의 학습 파이프라인을 구현할 차례입니다.<br>
`linear_classifer.py` 파일 안의 `train_linear_classifier` 함수를 완성하세요.  

잘 구현했다면 아래 코드 셀을 실행해 기본 하이퍼파라미터로 선형 분류기(linear classifier)를 학습해 보세요.<br>
(참고로 학습이 잘 되지 않는게 정상입니다.)

In [ ]:
import utils
from linear_classifier import softmax_loss_vectorized, train_linear_classifier

utils.reset_seed(0)

tic = time.time()

W, loss_hist = train_linear_classifier(softmax_loss_vectorized, None, 
                                       data_dict['X_train'], 
                                       data_dict['y_train'], 
                                       learning_rate=1e-10, reg=2.5e4,
                                       num_iters=1500, verbose=True)

toc = time.time()
print('That took %fs' % (toc - tic))

### 구현 하기

학습 코드는 작성했으니 이제 prediction 단계를 구현합니다.
학습된 모델을 training set과 validation set에서 평가해 보세요.  
위에서 학습한 모델은 제대로 학습이 되지 않았기 때문에 결과로 나오는 **validation accuracy**는 10% 미만이어야 합니다.  

In [ ]:
import utils
from linear_classifier import predict_linear_classifier

# fix random seed before we perform this operation
utils.reset_seed(0)

# evaluate the performance on both the training and validation set
# YOUR_TURN: Implement how to make a prediction with the trained weight 
#            in 'predict_linear_classifier'
y_train_pred = predict_linear_classifier(W, data_dict['X_train'])
train_acc = 100.0 * (data_dict['y_train'] == y_train_pred).double().mean().item()
print('Training accuracy: %.2f%%' % train_acc)

y_val_pred = predict_linear_classifier(W, data_dict['X_val'])
val_acc = 100.0 * (data_dict['y_val'] == y_val_pred).double().mean().item()
print('Validation accuracy: %.2f%%' % val_acc)

### 구현 하기 & 실험 하기
참고로 validation accuracy가 10%인 경우는 랜덤으로 찍는 것과 동일한 결과입니다...

그럼 이제 최적의 하이퍼파라미터(hyperparameter)를 찾아 좀 더 좋은 모델 성능을 기록해봅시다.<br>
아래 코드 셀은 validation dataset을 사용해서 hyperparameter tuning을 진행합니다.<br>
여기서 다양한 하이퍼파라미터를 탐색해보세요. 특히 regularization strength와 learning rate가 중요합니다.<br>

**본 과제에서는 `validation accuracy`가 37%를 넘는 결과를 한 번이라도 만들어야 100점으로 인정합니다.** 다양한 하이퍼파라미터 탐색을 진행해보세요.

In [ ]:
import os
import utils
from linear_classifier import Softmax, softmax_get_search_params, test_one_param_set

# TODO: find the best learning_rates and regularization_strengths combination in 'softmax_get_search_params'
learning_rates, regularization_strengths = softmax_get_search_params()
num_models = len(learning_rates) * len(regularization_strengths)

i = 0
# As before, store your cross-validation results in this dictionary.
# The keys should be tuples of (learning_rate, regularization_strength) and
# the values should be tuples (train_acc, val_acc)
results = {}
best_val = -1.0   # The highest validation accuracy that we have seen so far.
best_softmax_model = None # The Softmax object that achieved the highest validation rate.
num_iters = 2000 # number of iterations

for lr in learning_rates:
  for reg in regularization_strengths:
    i += 1
    print('Training Softmax %d / %d with learning_rate=%e and reg=%e'
          % (i, num_models, lr, reg))
    
    utils.reset_seed(0)
    cand_softmax_model, cand_train_acc, cand_val_acc = test_one_param_set(Softmax(), data_dict, lr, reg, num_iters)

    if cand_val_acc > best_val:
      best_val = cand_val_acc
      best_softmax_model = cand_softmax_model # save the classifier
    results[(lr, reg)] = (cand_train_acc, cand_val_acc)


# Print out results.
for lr, reg in sorted(results):
  train_acc, val_acc = results[(lr, reg)]
  print('lr %e reg %e train accuracy: %f val accuracy: %f' % (
         lr, reg, train_acc, val_acc))
    
print('best validation accuracy achieved during cross-validation: %f' % best_val)

아래 코드는 하이퍼파라미터 탐색 결과를 시각화합니다.

아래 코드는 학습한 최고의 모델을 테스트 데이터셋에 테스트합니다.

In [ ]:
y_test_pred = best_softmax_model.predict(data_dict['X_test'])
test_accuracy = torch.mean((data_dict['y_test'] == y_test_pred).double())
print('softmax on raw pixels final test set accuracy: %f' % (test_accuracy, ))

아래 코드 셀은 학습한 linear classifier의 weight를 시각화 합니다.

In [ ]:
w = best_softmax_model.W
w = w.reshape(3, 32, 32, 10)
w = w.transpose(0, 2).transpose(1, 0)

w_min, w_max = torch.min(w), torch.max(w)

classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
for i in range(10):
  plt.subplot(2, 5, i + 1)

  # Rescale the weights to be between 0 and 255
  wimg = 255.0 * (w[:, :, :, i].squeeze() - w_min) / (w_max - w_min)
  plt.imshow(wimg.type(torch.uint8).cpu())
  plt.axis('off')
  plt.title(classes[i])